The aim of this notebook will be to find the following data:
* decision tree to plot in the visualization
* the generated neighbourhood generated by LORE

Everything will be tested on the iris dataset for quick running times

### dataset import

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import json
from sklearn.preprocessing import LabelEncoder

In [2]:
data_file = "iris.json"
# Load and prepare data
data = load_iris()
X = data.data
y = data.target
feature_names = list(data.feature_names)
target_names = list(data.target_names)

### LORE initial

read data

In [3]:
from lore_sa.dataset import TabularDataset
import pandas as pd

In [4]:
data_dict = {name: X[:, i] for i, name in enumerate(feature_names)}
target_name = 'target'
data_dict[target_name] = [target_names[i] for i in y]  # Map numerical targets to names

dataset = TabularDataset.from_dict(data_dict, 'target')
dataset.df.dropna(inplace = True)
dataset.df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


train model

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lore_sa.bbox import sklearn_classifier_bbox

In [6]:

def tutorial_train_model_generalized(dataset: TabularDataset, target_name:str, ):
    numeric_indices = [v['index'] for k, v in dataset.descriptor['numeric'].items()]
    categorical_indices = [v['index'] for k, v in dataset.descriptor['categorical'].items()]

    # Create preprocessor using dynamic indices
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_indices),
            ('cat', OrdinalEncoder(), categorical_indices)
        ]
    )

    # Remove rare classes with fewer than 2 instances
    valid_classes = dataset.df[target_name].value_counts()[dataset.df[target_name].value_counts() > 1].index
    dataset.df = dataset.df[dataset.df[target_name].isin(valid_classes)]

        # Select features and target
    X = dataset.df.iloc[:, numeric_indices + categorical_indices]  # Select all features
    y = dataset.df[target_name]

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                test_size=0.3, random_state=42, stratify=y)

    model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, random_state=42))
    
    model.fit(X_train, y_train)
    
    return sklearn_classifier_bbox.sklearnBBox(model)

In [7]:
bbox = tutorial_train_model_generalized(dataset, target_name)

### encoding decoding

In [8]:
from lore_sa.encoder_decoder import ColumnTransformerEnc

tabular_enc = ColumnTransformerEnc(dataset.descriptor)
ref_value = dataset.df.iloc[0].values[:-1]
encoded = tabular_enc.encode([ref_value])
decoded = tabular_enc.decode(encoded)

print(f"Original value: {ref_value}")
print(f"Encoded value: {encoded}")
print(f"Decoded value: {decoded}")

Original value: [5.1 3.5 1.4 0.2]
Encoded value: [[5.1 3.5 1.4 0.2]]
Decoded value: [[5.1 3.5 1.4 0.2]]


the encoder is created using the dataset.descriptor, which is a dict so it can be saved to json and the encoder is then created based on the content read from it

In [9]:
dataset.descriptor

{'numeric': {'sepal length (cm)': {'index': 0,
   'min': 4.3,
   'max': 7.9,
   'mean': 5.843333333333334,
   'std': 0.828066127977863,
   'median': 5.8,
   'q1': 5.1,
   'q3': 6.4},
  'sepal width (cm)': {'index': 1,
   'min': 2.0,
   'max': 4.4,
   'mean': 3.0573333333333337,
   'std': 0.4358662849366982,
   'median': 3.0,
   'q1': 2.8,
   'q3': 3.3},
  'petal length (cm)': {'index': 2,
   'min': 1.0,
   'max': 6.9,
   'mean': 3.7580000000000005,
   'std': 1.7652982332594662,
   'median': 4.35,
   'q1': 1.6,
   'q3': 5.1},
  'petal width (cm)': {'index': 3,
   'min': 0.1,
   'max': 2.5,
   'mean': 1.1993333333333336,
   'std': 0.7622376689603465,
   'median': 1.3,
   'q1': 0.3,
   'q3': 1.8}},
 'categorical': {},
 'ordinal': {},
 'target': {'target': {'index': 4,
   'distinct_values': ['setosa', 'versicolor', 'virginica'],
   'count': {'setosa': 50, 'versicolor': 50, 'virginica': 50}}}}

the value to use for the prediction can be found via:

In [10]:
ref_value = dataset.df.iloc[0].values[:-1]
list(ref_value)

[5.1, 3.5, 1.4, 0.2]

and decoded using

In [11]:
encoded = tabular_enc.encode([ref_value])
decoded = tabular_enc.decode(encoded)
list(decoded[0])

[5.1, 3.5, 1.4, 0.2]

### neighborhood generation

In [12]:
from lore_sa.neighgen import RandomGenerator

select the istance to explain

In [13]:
num_row = 0
x = dataset.df.iloc[num_row][:-1]
x

sepal length (cm)    5.1
sepal width (cm)     3.5
petal length (cm)    1.4
petal width (cm)     0.2
Name: 0, dtype: object

encode it

In [14]:
z = tabular_enc.encode([x.values])[0] # remove the class feature from the input instance

creates the neighborhood

In [15]:
gen = RandomGenerator(bbox=bbox, dataset=dataset, encoder=tabular_enc, ocr=0.1)
neighbour = gen.generate(z, 100, dataset.descriptor, tabular_enc)
neighbour

array([[5.1, 3.5, 1.4, 0.2],
       [5.1, 3.5, 1.4, 0.2],
       [5.1, 3.5, 1.4, 0.2],
       [5.1, 3.5, 1.4, 0.2],
       [5.1, 3.5, 1.4, 1.0656360057812635],
       [5.1, 3.5, 1.4, 1.0656360057812635],
       [5.1, 3.5, 1.4, 1.0656360057812635],
       [5.1, 3.5, 1.4, 1.0656360057812635],
       [5.1, 3.5, 1.4, 1.0656360057812635],
       [5.1, 3.5, 6.583900700973793, 1.0656360057812635],
       [5.256847365452707, 3.5, 6.583900700973793, 1.0656360057812635],
       [5.256847365452707, 3.5, 6.583900700973793, 1.0656360057812635],
       [5.256847365452707, 2.528978129381757, 6.583900700973793,
        1.0656360057812635],
       [5.256847365452707, 2.528978129381757, 6.583900700973793,
        1.0656360057812635],
       [5.256847365452707, 2.528978129381757, 6.583900700973793,
        1.0656360057812635],
       [5.256847365452707, 2.528978129381757, 6.583900700973793,
        1.0656360057812635],
       [5.256847365452707, 2.528978129381757, 6.583900700973793,
        1.06563600578

### surrogate model

In [16]:
from lore_sa.surrogate import DecisionTreeSurrogate

create X, y and yz for the decision tree surrogate

In [17]:
# decode the neighborhood to be labeled by the blackbox model
neighb_train_X = tabular_enc.decode(neighbour)
neighb_train_y = bbox.predict(neighb_train_X)
# encode the target class to the surrogate model
neighb_train_yz = tabular_enc.encode_target_class(neighb_train_y.reshape(-1, 1)).squeeze()

train the surrogate model

In [ ]:
dt = DecisionTreeSurrogate()
x = dt.train(neighbour, neighb_train_yz)

get the decision tree _tree for plotting

In [18]:
dt.get_tree_structure()